In [0]:
# -*- coding: utf-8 -*-
#
# CNNによるクラス分類（MNIST）
#

In [0]:
# chainer のインストール
!curl https://colab.chainer.org/install | sh -

# 必要なライブラリイのインストール
!pip install h5py
!pip install pillow

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0   8307      0 --:--:-- --:--:-- --:--:--  8307
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-9-2 is already the newest version (9.2.148-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
+ pip install -q cupy-cuda92  chainer 
+ set +ex
Installation succeeded!


In [0]:
# drive のマウント(認証が必要)
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# データの展開
!unzip -q  '/content/gdrive/My Drive/data/mnist.zip'
!ls

gdrive	mnist  sample_data


In [0]:
import sys
import numpy as np
import chainer
from chainer import cuda
from chainer import Function
from chainer import report
from chainer import training
from chainer import utils
from chainer import Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList, cuda
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from PIL import Image

In [0]:
# chainerのバージョン
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
chainer.print_runtime_info()

GPU availability: True
cuDNN availablility: True
Platform: Linux-4.14.65+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 5.0.0
NumPy: 1.14.6
CuPy:
  CuPy Version          : 5.0.0
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 9020
  CUDA Driver Version   : 9020
  CUDA Runtime Version  : 9020
  cuDNN Build Version   : 7201
  cuDNN Version         : 7201
  NCCL Build Version    : 2213
iDeep: 2.0.0.post3


In [0]:
# CPU，GPUの表示
!cat /proc/cpuinfo
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
mode

In [0]:
# クラス数
class_num = 10

# 画像の大きさ
XSIZE = 28
YSIZE = 28

# 学習データ数
train_num = 100

# データ（GPUの場合，単精度の方が早い場合が多いです）
data_vec = np.zeros((class_num,train_num,YSIZE,XSIZE), dtype=np.float32)

# 学習のパラメータ
batchsize=10
n_epoch=10
n_train=class_num*train_num

In [0]:
# データの読み込み
def Read_data( flag ):

    dir = [ "train" , "test" ]
    for i in range(class_num):
        print( i )
        for j in range(1,train_num+1):
            # グレースケール画像で読み込み→大きさの変更→numpyに変換
            train_file = "mnist/" + dir[ flag ] + "/" + str(i) + "/" + str(i) + "_" + str(j) + ".jpg"
            work_img = Image.open(train_file).convert('L')
            data_vec[i][j-1]  = np.asarray(work_img).astype(np.float32)
            
            # データの正規化
            data_vec[i][j-1] = data_vec[i][j-1] / 255.0
    print( "----" )

In [0]:
# CNN
class CNN(chainer.Chain):
    # 畳み込みネットワークの設定
    def __init__(self):
        super(CNN, self).__init__(
            # 畳み込み層の設定
            conv1 = L.Convolution2D(1, 64, 3, stride=1, pad=1),
            conv2 = L.Convolution2D(64, 64, 3, stride=1, pad=1),
            conv3 = L.Convolution2D(64, 64, 3, stride=1, pad=1),
            conv4 = L.Convolution2D(64, 64, 3, stride=1, pad=1),
            conv5 = L.Convolution2D(64, 64, 3, stride=1, pad=1),
            conv6 = L.Convolution2D(64, 64, 3, stride=1, pad=1),

            # 全結合層の設定
            full1 = L.Linear(64*7*7,100),
            full2 = L.Linear(100, 10)
        )

    # 損失関数
    def __call__(self, x, y):
        # ソフトマックスクロスエントロピー誤差
        return F.softmax_cross_entropy(self.fwd(x), y)

    # 畳み込みネットワーク
    def fwd(self, x):
        # 畳み込み→畳み込み→プーリング
        h = F.relu(self.conv1(x))
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(h, ksize=2,stride=2,pad=0)

        # 畳み込み→畳み込み→プーリング
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.max_pooling_2d(h, ksize=2,stride=2,pad=0)

        # 畳み込み→畳み込み
        h = F.relu(self.conv5(h))
        h = F.relu(self.conv6(h))

        # 全結合層
        h = F.relu(self.full1(h))
        out = self.full2(h)
        return out

In [0]:
# 学習
def Train():
    # Adamによる更新
    optimizer = optimizers.Adam(alpha=0.0002, beta1=0.5)
    optimizer.setup(model)
    # 正則化
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.00001))
    
    for epoch in range(n_epoch):
        error = 0.0
        for i in range(0, n_train, batchsize):
            x = np.zeros((batchsize, 1, YSIZE, XSIZE), dtype=np.float32)
            y = np.zeros(batchsize, dtype=np.int32)

            # バッチの作成
            for j in range(batchsize):
                rnd_c = np.random.randint(class_num)
                rnd = np.random.randint(train_num)
                x[j,0,:,:] = data_vec[rnd_c][rnd]
                y[j] = rnd_c 

            # 入力データ
            xt = Variable(cuda.to_gpu(x))
            yt = Variable(cuda.to_gpu(y))

            # 勾配の初期化→伝播，誤差の計算→逆伝播→パラメータの更新
            model.zerograds()
            loss = model( xt , yt )
            error += loss.data
            loss.backward()
            optimizer.update()

            if i != 0 and i % 100 == 0:
               print( ' (' + str( epoch ) + ',' + str( i ) + ') : ' + str( error ) )
               error = 0.0

    # パラメータの保存
    serializers.save_hdf5("/content/gdrive/My Drive/data/model-CNN-gpu.h5", model)

In [0]:
# 予測
def Predict():

    # パラメータのロード
    serializers.load_hdf5("/content/gdrive/My Drive/data/model-CNN-gpu.h5", model)
    x = np.zeros((1, 1, YSIZE, XSIZE), dtype=np.float32)
    y = np.zeros( 1, dtype=np.int32)

    # 混合行列
    result = np.zeros((class_num,class_num), dtype=np.int32)
    
    for i in range(class_num):
        for j in range(train_num):
            # 入力データ
            x[0,0,:,:] = data_vec[i][j]
            y[0] = i
            
            xt = Variable(cuda.to_gpu(x))
            yt = Variable(cuda.to_gpu(y))
            
            # 予測
            predict = model.fwd( xt )
            ans = np.argmax( predict.data[0] )

            # 混合行列
            result[i][int(ans)] += 1

    print( "\n [混合行列]" )
    print( result )
    print( "\n 正解数 ->" ,  np.trace(result) )

In [0]:
# モデルの設定（GPU）
gpu_device = 0
cuda.get_device(gpu_device).use()
model = CNN()
model.to_gpu()


In [0]:
# データの読み込み
flag = 0
Read_data( flag )

# 学習
Train()

0
1
2
3
4
5
6
7
8
9
----
 (0,100) : 25.227198
 (0,200) : 22.04723
 (0,300) : 18.716139
 (0,400) : 10.432272
 (0,500) : 8.0285225
 (0,600) : 7.8106704
 (0,700) : 3.5015702
 (0,800) : 4.433708
 (0,900) : 5.2394414
 (1,100) : 3.606538
 (1,200) : 2.9365883
 (1,300) : 2.706869
 (1,400) : 3.311139
 (1,500) : 2.886369
 (1,600) : 1.4251643
 (1,700) : 1.4820572
 (1,800) : 2.6871595
 (1,900) : 3.4853873
 (2,100) : 1.2328894
 (2,200) : 0.4632244
 (2,300) : 2.0389476
 (2,400) : 2.305337
 (2,500) : 0.8456947
 (2,600) : 2.2104461
 (2,700) : 1.9521723
 (2,800) : 1.6576452
 (2,900) : 1.3926767
 (3,100) : 1.3879385
 (3,200) : 1.3633529
 (3,300) : 0.82640785
 (3,400) : 1.1129125
 (3,500) : 0.9763315
 (3,600) : 1.3389509
 (3,700) : 1.6211966
 (3,800) : 0.6289312
 (3,900) : 0.5314214
 (4,100) : 1.2321743
 (4,200) : 0.5435087
 (4,300) : 0.3815787
 (4,400) : 1.4626524
 (4,500) : 0.3988031
 (4,600) : 0.4487029
 (4,700) : 0.7943758
 (4,800) : 1.5022634
 (4,900) : 2.1678731
 (5,100) : 0.74208486
 (5,200) : 0.6

In [0]:
# データの読み込み
flag = 1
Read_data( flag )

# 予測
Predict()

0
1
2
3
4
5
6
7
8
9
----

 [混合行列]
[[ 99   0   0   0   0   0   1   0   0   0]
 [  0 100   0   0   0   0   0   0   0   0]
 [  0   0  96   0   0   0   1   1   2   0]
 [  0   0   0  93   0   6   0   0   1   0]
 [  0   0   0   0  94   0   2   0   1   3]
 [  1   0   0   0   0  94   1   0   4   0]
 [  3   1   0   0   0   2  94   0   0   0]
 [  0   1   1   8   1   1   0  86   0   2]
 [  2   0   1   1   1   0   1   0  93   1]
 [  0   0   0   0   0   1   0   1   2  96]]

 正解数 -> 945
